In [765]:
resource = "../../data/generated/"
results = "../../results/"

In [766]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS, compare
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from gmm import *
from linearmodels.asset_pricing import LinearFactorModelGMM
from tabprintin.beautify import *
from statsmodels.sandbox.regression import gmm

In [767]:
# Set the start and end dates of the analysis period
start_date = pd.to_datetime('1978-01-01')
# start_date = pd.to_datetime('1975-01-01')
end_date = pd.to_datetime('2008-04-30')

In [768]:
tables = []
plots = []

In [769]:
ts = pd.read_csv(resource + 'time_series.csv', parse_dates=['date'], index_col=['date'])
ts.index.freq = 'M'

# Compute the log change of industrial production over next 12 months (or just growth)
ts['log_indprod_growth_nextyear'] = np.log(ts['ind_prod'].shift(-12) / ts['ind_prod'].shift(-1))
# ts['indprod_growth_nextyear'] = ts['ind_prod'].shift(-12) / ts['ind_prod'].shift(-1) - 1

##################
## Base assets ###
##################

# Market return
# ts['ex_mkt'] = ts['ex_mkt'] /100
ts['lag_ex_mkt'] = ts['ex_mkt'].shift(1)

# Compute the excess return of the long-term government bond portfolio
ts['ex_long_gov_ret'] = ts['long_gov_ret'] - ts['rf']
ts['lag_ex_long_gov_ret'] = ts['ex_long_gov_ret'].shift(1)

# Compute the excess return of the intermediate-term government bond portfolio
ts['ex_medium_gov_ret'] = ts['medium_gov_ret'] - ts['rf']
ts['lag_ex_medium_gov_ret'] = ts['ex_medium_gov_ret'].shift(1)

# Compute the excess return of the high-yield bond portfolio
ts['ex_high_yd_bd_ret'] = ts['high_yd_bd_ret'] - ts['rf']
ts['lag_ex_high_yd_bd_ret'] = ts['ex_high_yd_bd_ret'].shift(1)

# Compute the return for gold index
ts['ex_gold_ret'] = ts['gold'].pct_change() - ts['rf']
ts['lag_ex_gold_ret'] = ts['ex_gold_ret'].shift(1)

# Create dummies for 1987 (stock market crash) and 1996-2002 (Internet bubble period)
ts['dummy_87'] = (ts.index.year == 1987).astype(int)
ts['dummy_96_02'] = ((ts.index.year >= 1996) & (ts.index.year <= 2002)).astype(int)

#########################
### Control Variables ###
#########################
# Compute the 10 year minus 3 month government bond yield
ts['lag_10y_3m_gov_bd_yd'] = (ts['DGS10'] - ts['DTB3']).shift(1)

# Compute the 1 year minus 3 month government bond yield
ts['lag_1y_3m_gov_bd_yd'] = (ts['DGS1'] - ts['DTB3']).shift(1)

# Baa minus Aaa corporate bond yield
ts['lag_Baa_Aaa_bd_yd'] = (ts['BAA'] - ts['AAA']).shift(1)

# Compute the dividend yield on the S&P 500 index
ts['lag_sp_div_yd'] = (ts['sp_div'] / ts['sp_price']).shift(1)

# Compute the log change of industrial production over last 12 months (or just growth)
ts['log_indprod_growth_lastyear'] = np.log(ts['ind_prod'].shift(13) / ts['ind_prod'].shift(1))
# ts['indprod_growth_lastyear'] = ts['ind_prod'].shift(13) / ts['ind_prod'].shift(1)  - 1


# Compute the inflation over last 12 months
# ts['infl_lastyear'] = (ts['cpi'].shift(1) - ts['cpi'].shift(13)) / ts['cpi'].shift(13)
ts['infl_lastyear'] = np.log(ts['cpi'].shift(1) / ts['cpi'].shift(13))

# Compute the market portfolio excess return over last 12 months
# [Controllare e sbagliato]
ts['ex_mkt_lastyear'] = (((ts['ex_mkt'] + 100)/100).rolling(13).apply(lambda x: x[:-1].prod()) - 1) * 100

# Interactions
ts['slope_ex_mkt_87'] = ts['ex_mkt'] * ts['dummy_87']
ts['slope_ex_mkt_9602'] = ts['ex_mkt'] * ts['dummy_96_02']

ts['lag_slope_ex_mkt_87'] = ts['slope_ex_mkt_87'].shift(1)
ts['lag_slope_ex_mkt_9602'] = ts['slope_ex_mkt_9602'].shift(1)

In [770]:
ts1 = ts.loc[(ts.index >= start_date) & (ts.index <= end_date)]

In [771]:
# mod1 = PanelOLS.from_formula('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_long_gov_ret+ex_medium_gov_ret+ex_high_yd_bd_ret+ex_gold_ret+ex_mkt*dummy_87+ex_mkt*dummy_96_02+lagged_10y_3m_gov_bd_yd+lagged_1y_3m_gov_bd_yd+lagged_Baa_Aaa_bd_yd+lagged_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear',
#                              data=ts)

mod1 = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_long_gov_ret+ex_medium_gov_ret+ex_high_yd_bd_ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear',
                data=ts1)

mod2 = smf.ols('log_indprod_growth_nextyear ~ 1+ex_mkt+ex_long_gov_ret+ex_medium_gov_ret+ex_high_yd_bd_ret+ex_gold_ret+slope_ex_mkt_87+slope_ex_mkt_9602+rf+lag_10y_3m_gov_bd_yd+lag_1y_3m_gov_bd_yd+lag_Baa_Aaa_bd_yd+lag_sp_div_yd+log_indprod_growth_lastyear+infl_lastyear+ex_mkt_lastyear+ lag_ex_mkt+lag_ex_long_gov_ret+lag_ex_medium_gov_ret+lag_ex_high_yd_bd_ret+lag_ex_gold_ret+lag_slope_ex_mkt_87+lag_slope_ex_mkt_9602',
                data=ts1)

reg1 = mod1.fit(cov_type='HAC',cov_kwds={'maxlags':11})
reg2 = mod2.fit(cov_type='HAC',cov_kwds={'maxlags':11})

tables.append(reg1)
tables.append(reg2)

Summary statistics

In [772]:
# Get the one-year ahead industrial production growth expectatitions factor
coef = reg2.params
# ts['myp'] = coef['ex_mkt'] * ts['ex_mkt'] + coef['ex_long_gov_ret'] * ts['ex_long_gov_ret'] + coef['ex_medium_gov_ret'] * ts['ex_medium_gov_ret'] + coef['ex_high_yd_bd_ret'] * ts['ex_high_yd_bd_ret'] + coef['ex_gold_ret'] * ts['ex_gold_ret'] + coef['slope_ex_mkt_87'] * ts['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * ts['slope_ex_mkt_9602']
def compute_myp(row):
    if pd.isna(row['ex_high_yd_bd_ret']):
        return coef['ex_mkt'] * row['ex_mkt'] + coef['ex_long_gov_ret'] * row['ex_long_gov_ret'] + coef['ex_medium_gov_ret'] * row['ex_medium_gov_ret'] + coef['ex_gold_ret'] * row['ex_gold_ret'] + coef['slope_ex_mkt_87'] * row['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * row['slope_ex_mkt_9602']
    else:
        return coef['ex_mkt'] * row['ex_mkt'] + coef['ex_long_gov_ret'] * row['ex_long_gov_ret'] + coef['ex_medium_gov_ret'] * row['ex_medium_gov_ret'] + coef['ex_high_yd_bd_ret'] * row['ex_high_yd_bd_ret'] + coef['ex_gold_ret'] * row['ex_gold_ret'] + coef['slope_ex_mkt_87'] * row['slope_ex_mkt_87'] + coef['slope_ex_mkt_9602'] * row['slope_ex_mkt_9602']

ts['myp'] = ts.apply(compute_myp, axis=1)

# Get the unexpected inflation factor
ts['infl'] = np.log(ts['cpi'] / ts['cpi'].shift(1))
ts['delta_infl'] = ts['infl'] - ts['infl'].shift(1)
inf_ma1 = sm.tsa.arima.ARIMA(ts['delta_infl'], order=(0,0,1)).fit()
ts['fit_delta_infl'] = inf_ma1.fittedvalues
ts['ui'] = ts['delta_infl'] - ts['fit_delta_infl']

# Get the change in the aggregate survival probability factor
mod3 = smf.ols('v_dsv ~ 1 + my_dsv', data=ts).fit()
parm = mod3.params
ts['fit_dsv'] = parm['Intercept'] + parm['my_dsv'] * ts['my_dsv']
ts['dsv'] = np.where(ts['v_dsv'].notna(), ts['v_dsv'], ts['fit_dsv'])

# Get the change in the average level of the term structure factor
ts['mean_term_structure'] = ts[['DTB3', 'DGS10']].mean(axis=1)
ts['ats'] = ts['mean_term_structure'] - ts['mean_term_structure'].shift(1)

# Get the change in the slope of the term strucutre factor
ts['diff_term_structure'] = ts['DGS10'] - ts['DTB3']
ts['sts'] = ts['diff_term_structure'] - ts['diff_term_structure'].shift(1)

# Get the change in the multilateral US dollar exchange rate factor
mod4 = smf.ols('TWEXM ~ 1 + DTWEXAFEGS', data=ts).fit()
parm = mod4.params
ts['fit_TWEXM'] = parm['Intercept'] + parm['DTWEXAFEGS'] * ts['DTWEXAFEGS']
ts['exchange_rate'] = np.where(ts['TWEXM'].notna(), ts['TWEXM'], ts['fit_TWEXM'])
ts['fx'] = ts['exchange_rate'] - ts['exchange_rate'].shift(1)
# ts['fx'] = np.log(ts['exchange_rate'] / ts['exchange_rate'].shift(1))

In [773]:
# Set the start and end dates of the analysis period
# start_date = pd.to_datetime('1975-01-01') ##############
# end_date = pd.to_datetime('1999-12-31') ##############
ts2 = ts.loc[(ts.index >= start_date) & (ts.index <= end_date)]

In [774]:
summary_stats = ts2[['myp','ui','dsv','ats','sts','fx']].describe().T  
tables.append(summary_stats)

Granger causality

In [775]:
# Create VAR model with constant term
model = VAR(ts2[['hml','smb','mom','myp','ui','dsv','ats','sts','fx']])
var = model.fit(maxlags=1, trend='c')

tables.append(var)

GMM

In [776]:
premia_port = []
premia_t_stat_port = []
beta_port = []
beta_t_stat_port = []

# For book-to-market portfolios
for index,ports in enumerate(['bm','size','mom']):
    ports_data = pd.read_csv(resource + f'{ports}_port.csv', parse_dates=['date'], index_col=['date'])
    ts3 = pd.merge(ts, ports_data, on='date', how='left')

    # Set the start and end dates of the analysis period
    # start_date = pd.to_datetime('1984-01-01') ##############
    # end_date = pd.to_datetime('1999-12-31') ##############
    ts3 = ts3.loc[(ts3.index >= start_date) & (ts3.index <= end_date)]

    macro_factors = ts3[['myp','ui','dsv','ats','sts','fx']].values
    financial_factors = ts3[['ex_mkt','smb','hml','mom']].values
    riskfree = ts3['rf'].values
    portfolios = ts3[['dec_1','dec_2','dec_3','dec_4','dec_5','dec_6','dec_7','dec_8','dec_9','dec_10']].values

    T,N = portfolios.shape
    excessRet = portfolios - np.reshape(riskfree,(T,1))
    K = np.size(macro_factors,1)

    # Starting values for the factor loadings and rick premia are estimated using OLS and simple means.
    betas = []
    for i in range(N):
        res = sm.OLS(excessRet[:,i],sm.add_constant(macro_factors)).fit()
        betas.append(res.params[1:])

    avgReturn = excessRet.mean(axis=0)
    avgReturn.shape = N,1
    betas = array(betas)
    res = sm.OLS(avgReturn, betas).fit()
    riskPremia = res.params

    # The starting values are computed the first step estimates are found using the non-linear optimizer. The initial weighting matrix is just the identify matrix.
    riskPremia.shape = K
    startingVals = np.concatenate((betas.flatten(),riskPremia))

    Winv = np.eye(N*(K+1))
    args = (excessRet, macro_factors, Winv)
    iteration = 0
    functionCount = 0
    # step1opt = fmin_bfgs(gmm_objective, startingVals, args=args, callback=iter_print)
    step1opt = fmin_bfgs(gmm_objective, startingVals, args=args)

    # Here we look at the risk premia estimates from the first step (inefficient) estimates.
    premia = step1opt[-K:]
    premia = Series(premia,index=['myp','ui','dsv','ats','sts','fx'])
    # print('Annualized Risk Premia (First step)')
    # print(100 * premia)

    # Next the first step estimates are used to estimate the moment conditions which are in-turn used to estimate the optimal weighting matrix for the moment conditions. This is then used as an input for the 2nd-step estimates.
    out = gmm_objective(step1opt, excessRet, macro_factors, Winv, out=True)
    S = np.cov(out[1].T)
    Winv2 = inv(S)
    args = (excessRet, macro_factors, Winv2)

    iteration = 0
    functionCount = 0
    # step2opt = fmin_bfgs(gmm_objective, step1opt, args=args, callback=iter_print)   
    step2opt = fmin_bfgs(gmm_objective, step1opt, args=args)  

    # The annualized risk premia.
    premia = step2opt[-K:]
    # premia = Series(premia,index=['myp','ui','dsv','ats','sts','fx'])
    # print('Annualized Risk Premia')
    # print(100 * premia)

    # Finally the VCV of the parameter estimates is computed.
    out = gmm_objective(step2opt, excessRet, macro_factors, Winv2, out=True)
    G = gmm_G(step2opt, excessRet, macro_factors)
    S = np.cov(out[1].T)
    vcv = inv(G @ inv(S) @ G.T)/T
    premia_vcv = vcv[-K:,-K:]
    premia_stderr = np.diag(premia_vcv)
    # premia_stderr = Series(premia_stderr,index=['myp','ui','dsv','ats','sts','fx'])
    # print('t-stats')
    # print(premia / premia_stderr)
    premia_t_stat = premia / premia_stderr

    beta = reshape(step2opt[:-K],(N,K))
    beta_vcv = vcv[:-K,:-K]
    beta_stderr = np.diag(beta_vcv)
    beta_t_stat = step2opt[:-K] / beta_stderr
    beta_t_stat = reshape(beta_t_stat,(N,K))

    premia_port.append(premia)
    premia_t_stat_port.append(premia_t_stat)
    beta_port.append(beta)
    beta_t_stat_port.append(beta_t_stat)


         Current function value: 0.000002
         Iterations: 19
         Function evaluations: 4166
         Gradient evaluations: 62
         Current function value: 1.167547
         Iterations: 136
         Function evaluations: 12674
         Gradient evaluations: 189
         Current function value: 0.000000
         Iterations: 12
         Function evaluations: 3830
         Gradient evaluations: 57
         Current function value: 2.098526
         Iterations: 95
         Function evaluations: 8923
         Gradient evaluations: 133
         Current function value: 0.000000
         Iterations: 12
         Function evaluations: 4233
         Gradient evaluations: 63
         Current function value: 0.228586
         Iterations: 140
         Function evaluations: 12541
         Gradient evaluations: 187


In [777]:
# # model = LinearFactorModelGMM(portfolios, macro_factors)
# model = LinearFactorModelGMM(excessRet, macro_factors, risk_free=False)

# # Estimate the model parameters
# results = model.fit(cov_type='kernel',bandwidth=12)

# # Print the summary of results
# print(results.full_summary)


In [778]:
# results.params
# results.tstats
# results.pvalues

In [779]:
# startingVals = np.zeros((1,K))
# Winv = np.eye(N)
# args = (excessRet, macro_factors, Winv)
# iteration = 0
# functionCount = 0
# opt_b = fmin_bfgs(gmm_objective_b, startingVals, args=args, callback=iter_print)
# sdf_loading = Series(opt_b,index=['myp','ui','dsv','ats','sts','fx'])
# sdf_loading

In [780]:
# # The GMM objective which needs to be minimized (to get factor loading b)
# def moment_b(params, fRets):
#     b = params
#     error = 1 - fRets @ b
#     return error

# def moment_consumption1(params, exog):
#     beta, gamma = params
#     r_forw1, c_forw1, c = exog.T  # unwrap iterable (ndarray)
    
#     # moment condition without instrument    
#     err = 1 - beta * (1 + r_forw1) * np.power(c_forw1 / c, -gamma)
#     return -err

# endog1 = np.zeros(macro_factors.shape[0])    
# mod10 = gmm.NonlinearIVGMM(endog1, macro_factors, excessRet, moment_b)
# w0inv = np.eye(N)
# res10 = mod10.fit(inv_weights=w0inv, maxiter=100, weights_method='hac', wargs={'maxlag':4}) 
# print(res10.summary())